# Import libraries

In [ ]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

import cv2
import os
import math
from math import ceil, floor, log

# to be used to get better performance
# from sklearn.model_selection import KFold
!pip install -q efficientnet
import efficientnet.tfkeras as efn

import scikitplot as skplt
from sklearn.utils import shuffle
print("TF version:", tf.__version__)
print(tf.config.list_physical_devices('GPU'))

# Preparations
Initialize hyperparameters

In [ ]:
N_epochs = 30
SEED = 1970
N_TTA = 5
# in the version 9 of the botebook I made images size = 300 for all models VGG16 19 RESTNET50 效果不好
MODELS = {
          'Xception':[tf.keras.applications.Xception,32,300],
          'MobileNet':[tf.keras.applications.MobileNet,32,300],
          'InceptionV3':[tf.keras.applications.InceptionV3,32,300],
          'DenseNet121':[tf.keras.applications.DenseNet121,32,300],
          'EfficientNetB3':[efn.EfficientNetB3,8,300],
          'EfficientNetB6':[efn.EfficientNetB6,8,300]
         }
path = '../input/'
path_hands = 'hand-palm/hand_split/' 
df_hands = pd.read_csv(path + path_hands + "hands1000_with_5_0.csv")
df_hands = shuffle(df_hands)
df_hands.head(5)

In [ ]:
dic=df_hands["label0"].unique()
df_hands["label"]=df_hands['label0']
df_hands["id"]=df_hands['photo_id']
df_hands['id'] = df_hands.id.apply(lambda x: int((x).replace('Hand_', '').replace(".jpg", "")) )
df_hands.head()

In [ ]:
df_hands_label_one_hot=pd.get_dummies(df_hands['label0'])
df_hands=df_hands.join(df_hands_label_one_hot)
pd.options.display.max_columns=50
df_hands.head(5)

Creating all dataframes

In [ ]:


#df_hands['path'] = df_flowers.path.apply(lambda x: x.replace('\\', '/') )
df_hands['path']=path + path_hands
df_hands['path'] = path + path_hands+df_hands['label'].astype(str)+"/"+ df_hands['photo_id'].astype(str) 

# generate file lists and labels
labels_hands_cols = df_hands['label'].unique().tolist()


df_valid = df_hands.sample(frac = 0.2)
 
df_train = df_hands.drop(df_valid.index)

df_test = df_valid.sample(frac = 0.5)
df_valid = df_valid.drop(df_test.index)
print (len(df_valid),len(df_test),len(df_train))
y_train = df_train.loc[:,['id']+labels_hands_cols]


In [ ]:
#指定index 為id inplace 表示同時取代舊的data set 
y_train.set_index('id', inplace = True)
y_valid = df_valid.loc[:,['id']+labels_hands_cols]
y_valid.set_index('id', inplace = True)
y_test = df_test.loc[:,['id']+labels_hands_cols]
y_test.set_index('id', inplace = True)
df_train.head(5)

Defining learning rate scheduler and early stopping callbacks

In [ ]:
def get_lr_callback(batch_size = 16, plot=False):
    start_lr = 0.001
    def step_decay(epoch):
        drop = 0.5
        epochs_drop = 5.0
        lr = start_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
        return lr
    
    lr_callback = tf.keras.callbacks.LearningRateScheduler(step_decay)
    if plot == True:
        rng = [i for i in range(N_epochs)]
        y = [step_decay(x) for x in rng]
        plt.plot(rng, y)
        plt.xlabel('epoch', size=14)
        plt.ylabel('learning_rate', size=14)
        plt.title('Training Schedule', size=16)
        plt.show()
        
    return lr_callback


es_callback = tf.keras.callbacks.EarlyStopping(patience=10, 
                                               monitor='val_loss',
                                               verbose=1, 
                                               restore_best_weights=True)
lr = get_lr_callback(plot=True)

In [ ]:
def gen_init(BS, IMG_Size):
    train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        vertical_flip = True,
        horizontal_flip=True)

    valid_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)    
    train_generator = train_gen.flow_from_dataframe(dataframe=df_train, directory=path, 
                                                  x_col="path", y_col=labels_hands_cols, 
                                                  class_mode="raw", 
                                                  target_size=(IMG_Size,IMG_Size), batch_size = BS)

    valid_generator = valid_gen.flow_from_dataframe(dataframe=df_valid, directory=path, 
                                                  x_col="path", y_col=labels_hands_cols, 
                                                  class_mode="raw", 
                                                  # class_mode="categorical", 
                                                  target_size=(IMG_Size,IMG_Size), batch_size = BS)

    test_generator = valid_gen.flow_from_dataframe(dataframe=df_test, directory=path, 
                                                  x_col="path", y_col=labels_hands_cols, 
                                                  class_mode="raw", 
                                                  shuffle = False,
                                                  target_size=(IMG_Size,IMG_Size), batch_size = BS)
    return   train_generator,   valid_generator, test_generator

Define data generators

In [ ]:
train_generator, valid_generator, test_generator = gen_init(32,200)

In [ ]:
ti, tl = train_generator.next()
imgs = []
for i in range(ti.shape[0]):
    img = np.array(ti[i]*255, dtype = 'int32')
    imgs.append(img)

f, ax = plt.subplots(4, 8, figsize=(15,10))
for i, img in enumerate(imgs):
    ax[i//8, i%8].imshow(img)
    ax[i//8, i%8].axis('off')
    ax[i//8, i%8].set_title('label: %s' % tl[i])
plt.show()

# Main part
Build a basic model template, so we can try a few different engines from the MODEL dictionary

In [ ]:
def build_model(model_engine, IMG_Size):
    inp = tf.keras.layers.Input(shape=(IMG_Size,IMG_Size,3))
    base = model_engine(input_shape=(IMG_Size,IMG_Size,3),weights='imagenet',include_top=False)
    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(25,activation='softmax')(x)
    model = tf.keras.Model(inputs=inp,outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate = 0.001)   
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
all_model = []
all_history = []
all_preds = []
all_accuracies = []
all_confusion_matrices = []

In [ ]:
for model_name in MODELS:
    engine = MODELS[model_name][0]
    BS = MODELS[model_name][1]
    IMG_Size = MODELS[model_name][2]
    train_generator, valid_generator, test_generator = gen_init(BS, IMG_Size)
    
    model = build_model(engine, IMG_Size)
    print('------------------------------------------------------------------')
    print('Training model ', model_name)
    history = model.fit(train_generator,
              steps_per_epoch=len(df_train) / BS, epochs = N_epochs, verbose = 1,
              callbacks=[es_callback, get_lr_callback(BS)],
              validation_data = valid_generator)

    model.save('model-%s.h5'%model_name)  
    all_history.append(history)
    
    pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot()
    pd.DataFrame(history.history)[['loss', 'val_loss']].plot()
    plt.show()

    preds = model.predict(test_generator, verbose = 1)
    all_preds.append(preds)
    cm = confusion_matrix(np.argmax(np.array(y_test), axis=1), np.argmax(preds, axis = 1))
    all_confusion_matrices.append(cm)
    acc = accuracy_score(np.argmax(np.array(y_test), axis=1), np.argmax(preds, axis = 1))    
    all_accuracies.append(acc)
    print('------------------------------------------------------------------')
    print(cm)
    print(acc)
    

# Analysis
Lets have a look at all of the confusion matrices

In [ ]:
for i, model_name in enumerate(MODELS):
    skplt.metrics.plot_confusion_matrix(
        np.argmax(np.array(y_test), axis=1), np.argmax(all_preds[i], axis = 1),
        figsize=(8,8))

And now for an alternative view here all these matrices are normalized.

In [ ]:
for i, model_name in enumerate(MODELS):
    skplt.metrics.plot_confusion_matrix(
        np.argmax(np.array(y_test), axis=1), np.argmax(all_preds[i], axis = 1), normalize=True,
        figsize=(8,8))

In [ ]:
#因為左右手分開處理 避免混亂所 以不做水平翻轉
def gen_test_init(df, BS, IMG_Size):
    tta_gen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        vertical_flip =  True,
        horizontal_flip=False)

    valid_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)    

    test_generator = valid_gen.flow_from_dataframe(dataframe=df, directory=path, 
                                                  x_col="path", y_col=labels_hands_cols, 
                                                  class_mode="raw", 
                                                  shuffle = False,
                                                  target_size=(IMG_Size,IMG_Size), batch_size = BS)
    tta_test_generator = tta_gen.flow_from_dataframe(dataframe=df, directory=path, 
                                                  x_col="path", y_col=labels_hands_cols, 
                                                  class_mode="raw", 
                                                  shuffle = False,
                                                  target_size=(IMG_Size,IMG_Size), batch_size = BS)
    return  tta_test_generator, test_generator

In [ ]:
all_preds_tta = []
all_accuracies_tta = []
all_confusion_matrices_tta = []

model_name = 'EfficientNetB6'
engine = MODELS[model_name][0]
BS = MODELS[model_name][1]
IMG_Size = MODELS[model_name][2]
tta_test_generator, test_generator = gen_test_init(df_test, BS, IMG_Size)

model = tf.keras.models.load_model('model-%s.h5'%model_name)
print('Predicting original images', model_name)

preds_org = model.predict(test_generator, verbose = 1)
all_preds_tta.append(preds_org)
cm_org = confusion_matrix(np.argmax(np.array(y_test), axis=1), np.argmax(preds_org, axis = 1))
all_confusion_matrices_tta.append(cm_org)
acc_org = accuracy_score(np.argmax(np.array(y_test), axis=1), np.argmax(preds_org, axis = 1))    
all_accuracies_tta.append(acc_org)
print(cm_org)
print(acc_org)

for i in range(N_TTA):
    print('Predicting images TTA %i for %s'%(i, model_name))

    preds_tta = model.predict(tta_test_generator, verbose = 1)
    all_preds_tta.append(preds_tta)
    cm_tta = confusion_matrix(np.argmax(np.array(y_test), axis=1), np.argmax(preds_tta, axis = 1))
    all_confusion_matrices_tta.append(cm_tta)
    acc_org = accuracy_score(np.argmax(np.array(y_test), axis=1), np.argmax(preds_tta, axis = 1))    
    all_accuracies_tta.append(acc_org)
    print(cm_tta)
    print(acc_org)

avg_preds = np.mean(np.array(all_preds_tta),axis = 0)
cm_tta = confusion_matrix(np.argmax(np.array(y_test), axis=1), np.argmax(avg_preds, axis = 1))
all_confusion_matrices_tta.append(cm_tta)
acc_org = accuracy_score(np.argmax(np.array(y_test), axis=1), np.argmax(avg_preds, axis = 1))    
all_accuracies_tta.append(acc_org)
print('Confusion matrix with %i TTA : '%N_TTA)
print(cm_tta)
print('Prediction with TTA - averaged accuracy %.4f' %acc_org) 
